Criteria description:
* **The total financial cost (criterion C1)** = This criterion represents the total of the expenses which are necessary to cover all the technical costs incurred before, during and after dumping of the radioactive wastes
* **The "polluter-pays principle" (criteria C2 z and C3)**, it corresponds to the electronuclear production generating the wastes to be buried into the repository:
    - **C2** represents the costs incurred by the present consumers;
    - **C3** = costs to be supported by the future consumers (although they did not take advantage of the t = 0 -> 30 year electronuclear production)
* **Financial risk due to overcosts (C4)**: related to a wrong assessment: for example, in the case of the kWh-financing
(F1), the risk is to be short of payments after the thirty year fee raising for the fund

All criteria are cost-type.

In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import *

### Setup

In [220]:
data = pd.read_csv("nuclear-waste-management.csv", index_col=0)
data

,C1,C2,C3,C4
1,0.60,0.93,0.00,0.73
2,0.66,0.55,0.45,0.49
3,1.00,0.45,0.57,0.50
4,0.48,0.87,0.00,0.75
5,0.62,0.40,0.56,0.50
6,0.78,0.27,0.71,0.50
7,0.40,0.90,0.00,0.82
8,0.64,0.44,0.54,0.54
9,0.65,0.30,0.71,0.55
10,0.45,0.86,0.00,0.73


In [221]:
u = pd.DataFrame.from_dict({
    f"C{criterion}": {
        u_arg: LpVariable(
            name=f"{criterion}_{u_arg}", lowBound=0, upBound=1, cat="Continuous"
        )
        for u_arg in range(101)
    }
    for criterion in range(1, 5)
})

u

,C1,C2,C3,C4
0,1_0,2_0,3_0,4_0
1,1_1,2_1,3_1,4_1
2,1_2,2_2,3_2,4_2
3,1_3,2_3,3_3,4_3
4,1_4,2_4,3_4,4_4
...,...,...,...,...
96,1_96,2_96,3_96,4_96
97,1_97,2_97,3_97,4_97
98,1_98,2_98,3_98,4_98
99,1_99,2_99,3_99,4_99


In [222]:
min_max = pd.DataFrame.from_dict({
    f"C{criterion}": {
        "best": min(data[f"C{criterion}"]),
        "worst": max(data[f"C{criterion}"]),
    }
    for criterion in range(1, 5)
})

min_max

,C1,C2,C3,C4
best,0.32,0.03,0.0,0.49
worst,1.00,1.00,1.0,1.00


In [223]:
def add_to_reference_ranking(model, a: int, b: int, epsilon: None | LpVariable = None) -> None:
    a_variant = (data.loc[a] * 100).astype(np.int8)
    b_variant = (data.loc[b] * 100).astype(np.int8)

    if epsilon is None:
        model.addConstraint(
            (
                u.C1[a_variant.C1] + u.C2[a_variant.C2] + u.C3[a_variant.C3] + u.C4[a_variant.C4]
                == u.C1[b_variant.C1] + u.C2[b_variant.C2] + u.C3[b_variant.C3] + u.C4[b_variant.C4]
            ),
            # f"#{a} indifferent to {b}",
        )
    else:
        model.addConstraint(
            (
                u.C1[a_variant.C1] + u.C2[a_variant.C2] + u.C3[a_variant.C3] + u.C4[a_variant.C4]
                >= u.C1[b_variant.C1] + u.C2[b_variant.C2] + u.C3[b_variant.C3] + u.C4[b_variant.C4] + epsilon
            ),
            # f"#{a} over {b}",
        )

In [224]:
def normalization_condition(model) -> None:
    limits = (min_max * 100).astype(np.uint8)

    model.addConstraint(
        (u.C1[limits.C1.best] + u.C2[limits.C2.best] + u.C3[limits.C3.best] + u.C4[limits.C4.best] == 1),
        "#normalization"
    )

    model.addConstraint(u.C1[limits.C1.worst] == 0, "#criterion 1 worst value")
    model.addConstraint(u.C2[limits.C2.worst] == 0, "#criterion 2 worst value")
    model.addConstraint(u.C3[limits.C3.worst] == 0, "#criterion 3 worst value")
    model.addConstraint(u.C4[limits.C4.worst] == 0, "#criterion 4 worst value")

In [225]:
def monotonicity_and_non_negativity_condition(model) -> None:  

    for criterion in data.columns:
        values = sorted(set((data[criterion] * 100).astype(np.int8)))

        for c_value in values:
            model += u[criterion][c_value] >= 0
        
        for i in range(1, len(values)):
            model += u[criterion][values[i - 1]] >= u[criterion][values[i]]

### UTA Method

In [226]:
def solver_main_function() -> LpProblem:
    model = LpProblem(name="nuclear-waste-management", sense=LpMaximize)
    epsilon = LpVariable(name="epsilon", lowBound=0, cat="Continuous")

    # constraints
    normalization_condition(model)
    monotonicity_and_non_negativity_condition(model)

    # reference ranking
    # 11 > 14 = 2 > 19 = 25
    # 10 > 19
    add_to_reference_ranking(model, 11, 14, epsilon)
    add_to_reference_ranking(model, 2, 19, epsilon)

    add_to_reference_ranking(model, 14, 2)

    add_to_reference_ranking(model, 10, 19, epsilon)
    add_to_reference_ranking(model, 25, 19)

    model += epsilon

    return model, epsilon

# solving
model, _ = solver_main_function()
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/annapralat/Studies/SI/ISWD/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hv/9h4vvcdx1px2mm2jfg7k1_0m0000gn/T/a1f260f2a4af40beb77b916e5471a848-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/hv/9h4vvcdx1px2mm2jfg7k1_0m0000gn/T/a1f260f2a4af40beb77b916e5471a848-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 163 COLUMNS
At line 435 RHS
At line 594 BOUNDS
At line 671 ENDATA
Problem MODEL has 158 rows, 77 columns and 269 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 45 (-113) rows, 45 (-32) columns and 113 (-156) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 7.2242825e-05 ( 0.0072242825%) - largest zero change 7.1142212e-05
0  Obj -0 Dual inf 0.99992676 (1)
31  Obj 0.9993679 Primal inf 1.999998 (2)
42  Obj 0.39963804
Optima

In [229]:
ranking = pd.DataFrame(data.copy())

for criterion in data.columns:
    ranking[f"U({criterion})"] = [0 for _ in range(27)]
ranking["U"] = [0 for _ in range(27)]

for criterion in data.columns:
    for variant in data.index:
        u_arg = (data[criterion][variant] * 100).astype(np.int8)
        ranking[f"U({criterion})"][variant] += round(u[criterion][u_arg].value(), 10)

for variant in data.index:
    ranking["U"][variant] = np.sum(ranking.loc[variant][5:])



ranking.sort_values('U', ascending=False)

/var/folders/hv/9h4vvcdx1px2mm2jfg7k1_0m0000gn/T/ipykernel_15807/2644740598.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranking[f"U({criterion})"][variant] += round(u[criterion][u_arg].value(), 10)
/var/folders/hv/9h4vvcdx1px2mm2jfg7k1_0m0000gn/T/ipykernel_15807/2644740598.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranking["U"][variant] = np.sum(ranking.loc[variant][5:])


,C1,C2,C3,C4,U(C1),U(C2),U(C3),U(C4),U
12,0.74,0.25,0.80,0.49,0.0,0.2,0,0.6,0.8
11,0.61,0.54,0.38,0.49,0.2,0.2,0,0.6,0.8
14,0.69,0.49,0.56,0.61,0.0,0.2,0,0.4,0.6
26,0.71,0.25,0.88,0.67,0.0,0.2,0,0.4,0.6
24,0.73,0.03,1.00,0.63,0.0,0.2,0,0.4,0.6
23,0.59,0.24,0.70,0.63,0.2,0.2,0,0.4,0.6
21,0.83,0.25,0.80,0.65,0.0,0.2,0,0.4,0.6
20,0.64,0.22,0.81,0.65,0.0,0.2,0,0.4,0.6
18,0.76,0.06,1.00,0.60,0.0,0.2,0,0.4,0.6
17,0.68,0.40,0.65,0.60,0.0,0.2,0,0.4,0.6


#### UTA GMS

In [230]:
def solver_uta_gms(a: int, b: int) -> LpProblem:
    model, epsilon = solver_main_function()

    add_to_reference_ranking(model, a, b, epsilon)

    status = model.solve()

    return model.objective.value()

epsilon_results = np.zeros((27, 27))
for i in range(epsilon_results.shape[0]):
    for j in range(epsilon_results.shape[1]):
        if i == j:
            continue
        result = solver_uta_gms(i + 1, j + 1)
        if result > 0:
            epsilon_results[i, j] = 1
        elif result == 0:
            epsilon_results[i, j] = 0
        else:
            epsilon_results[i, j] = -1

for i in range(epsilon_results.shape[0]):
    for j in range(epsilon_results.shape[1]):
        if i == j:
            continue
        if epsilon_results[i, j] >= 0 and epsilon_results[j, i] <= 0:
            print(f'necessary = ({i + 1}, {j + 1})')
        elif epsilon_results[i, j] >= 0:
            print(f'possible = ({i + 1}, {j + 1})')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/annapralat/Studies/SI/ISWD/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hv/9h4vvcdx1px2mm2jfg7k1_0m0000gn/T/89befc01041e46b09d5803ac9a8f848f-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/hv/9h4vvcdx1px2mm2jfg7k1_0m0000gn/T/89befc01041e46b09d5803ac9a8f848f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 164 COLUMNS
At line 445 RHS
At line 605 BOUNDS
At line 682 ENDATA
Problem MODEL has 159 rows, 77 columns and 278 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 50 (-109) rows, 49 (-28) columns and 133 (-145) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 0.00017696542 ( 0.017696542%) - largest zero change 0.00017286243
0  Obj -0 Dual inf 0.99982203 (1)
28  Obj 0.33268841
Optimal - objective value 0.33333333
After Postso